<a href="https://colab.research.google.com/github/Buenobarbie/GPT-from-scratch/blob/master/gpt_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Model

In [ ]:
MANUAL_SEED = 1337
LOAD = True
DATASET = None
import torch
import torch.nn as nn
import torch.nn.functional as F
import json

config_path = '../models/config.json'
model_path = '../models/model.pth'

save_config_path = '../models/config2.json'
save_model_path = '../models/mode2l.pth'


if LOAD:
    # load hyperparameters from json file
    with open(config_path) as f:
        data = json.load(f)
        batch_size = data['batch_size']
        block_size = data['block_size']
        max_steps = data['max_steps']
        learning_rate = data['learning_rate']
        eval_iters = data['eval_iters']
        n_emb = data['n_emb']
        n_layer = data['n_layer']
        n_head = data['n_head']
        dropout = data['dropout']
        vocab_size = data['vocab_size']
        stoi = {s:int(i) for s,i in data['stoi'].items()}
        itos = {int(i):s for i,s in data['itos'].items()}


else:
    # HYPERPARAMETERS ----------
    batch_size = 64
    block_size = 256
    max_steps = 2500
    learning_rate = 3e-4
    eval_iters = 200
    n_emb = 498
    n_layer = 6
    n_head = 6
    dropout = 0.2
    vocab_size = 469  
    # ------------------------
    
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(MANUAL_SEED)

## Preprocess dataset

In [ ]:
if DATASET:
    with open('../data/input.txt', 'r', encoding='utf-8') as f:
        lines = f.readlines()


    # Filter messages
    messages = [line[line.index('-')+2:] for line in lines if not("<Media omitted>" in line or "<This message was edited>" in line) and "-" in line] # Remove date

    # Join the messages with new line characters
    text = "".join(messages)
    print("Length of dataset in characters: ", len(text))
    print(text[:100])

In [ ]:
# Check all the unique characters in dataset
if not LOAD:
    chars = sorted(list(set(text)))
    vocab_size = len(chars)
    print("".join(chars))
    print("Vocabulary size: ", vocab_size)

In [ ]:
# Create a mapping of characters to integers
if not LOAD:
    stoi = {c:i for i,c in enumerate(chars)}
    itos = {i:c for i,c in enumerate(chars)}

In [ ]:
encode = lambda msg: [stoi[c] for c in msg]               # encode message
decode = lambda e_msg: "".join([itos[i] for i in e_msg])  # decode encoded message

print(encode("Hello, world!"))
print(decode(encode("Hello, world!")))

## Tokenize the entire dataset

In [ ]:
if DATASET:
    data = torch.tensor(encode(text), dtype=torch.long)
    print(data.shape, data.dtype)
    print(data[:10])

## Split dataset

In [ ]:
if DATASET:
    n = int(0.9 *len(data))
    train_data = data[:n]
    val_data = data[n:]

## Get chunck of data for training

In [ ]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))  # choose random starting points for each sequence in the batch
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x, y


## Head of self attention

In [ ]:
class Head(nn.Module):
    """One head of self attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_emb, head_size, bias=False)
        self.query = nn.Linear(n_emb, head_size, bias=False)
        self.value = nn.Linear(n_emb, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) # not a parameter of the model

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B, T, head_size)
        q = self.query(x) # (B, T, head_size)

        # compute attention scores
        wei = q @ k.transpose(-2, -1) * C**-0.5 # (B, T, head_size) @ (B, head_size, T) -> (B, T, T) # normalize ("scaled attention")
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        # perform weighted agregation of the values
        v = self.value(x)
        out = wei @ v

        return out

## Multi Head Attention

In [ ]:
class MultiHeadAttention(nn.Module):
    """Multiples heas of self-attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_emb, n_emb)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

## Feed Forward

In [ ]:
class FeedForward(nn.Module):
    """A simple linear layer with a non-linearity"""

    def __init__(self, n_emb):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_emb, 4*n_emb),
            nn.ReLU(),
            nn.Linear(4*n_emb, n_emb) ,# projection layer
            nn.Dropout(dropout)
        )

    def forward(self,x):
        return self.net(x)

## Block

In [ ]:
class  Block(nn.Module):
    """Transformer block: communication followed by computation"""

    def __init__(self,n_emb,n_head):
        # n_emb: embedding size
        # n_head: number of heads
        super().__init__()
        head_size = n_emb // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_emb)
        self.ln1 = nn.LayerNorm(n_emb)
        self.ln2 = nn.LayerNorm(n_emb)

    def forward(self,x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

## Language model

- B: Batch_size
- T: Length of the input sequence
- C: Number of features per token (size of the embedding)

In [ ]:

class LanguageModel(nn.Module):

    def __init__(self):
        super().__init__()

        # Initialize a matrix of shape (vocab_size, vocab_size) with  random values
        # that will be optimized during training
        self.token_embedding_table = nn.Embedding(num_embeddings=vocab_size, embedding_dim=n_emb)
        self.position_embedding_table = nn.Embedding(num_embeddings=block_size, embedding_dim=n_emb)
        self.blocks = nn.Sequential(*[Block(n_emb, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_emb) # final layer norm
        self.lm_head = nn.Linear(n_emb, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) # (B, T, n_emb)
        pos_emb = self.position_embedding_table(torch.arange(T,device=device)) # (T, n_emb)
        x = tok_emb + pos_emb # (B, T, n_emb)
        x = self.blocks(x)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None

        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indeces in the current context
        # the goal is to get (B, T+1), (B, T+2), ... (B, T+max_new_tokens )

        for _ in range(max_new_tokens):
            # crop the context to the last block_size tokens
            idx_crop = idx[:, -block_size:]

            # get the predictions
            logits, loss = self(idx_crop)

            # focus only on the last time step (the logit of the last token in the context)
            logits = logits[:, -1, :] # becomes (B, C)

            # softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)

            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)

            # append sampled index to the running sequence
            idx = torch.cat([idx, idx_next], dim=1)

        return idx



## Load the model

In [ ]:
if LOAD:
    model = LanguageModel()
    # Load the model state dictionary
    model.load_state_dict(torch.load(model_path, map_location=torch.device(device)))
else:
    m = LanguageModel()
    model = m.to(device)

## Train the Model

In [ ]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [ ]:
if DATASET:
    for steps in range(max_steps):

        # sample batch
        xb, yb = get_batch('train')

        # evaluate the loss
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    print(loss.item())

## Better estimate the loss

Since the loss is calculated for a single batch, it may not representate well the loss of the whole data.

So to estimate we'll calculate the loss for many batches and then return the mean.

In [ ]:
@torch.no_grad()
def estimate_loss(eval_iters=500):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            xb, yb = get_batch(split)
            logits, loss = model(xb, yb)
            losses[k] = loss.item()

        out[split] = losses.mean().item()
    model.train()
    return out

if DATASET:
    print(estimate_loss(eval_iters))

## Sample

In [ ]:
prompt = "\n"
prompt_encoded = encode(prompt)
idx = torch.tensor([prompt_encoded], dtype=torch.long, device=device) # Contains the character corresponding to index 0 (newline)
# idx.shape
print(decode(model.generate(idx, max_new_tokens=500)[0].tolist())) # decode the first batch of the generated completions

## Save the model

In [ ]:
SAVE = False

In [ ]:
if SAVE:
    torch.save(m.state_dict(), save_model_path)

In [ ]:
if SAVE:
    import json
    config = {
        "batch_size": batch_size,
        "block_size": block_size,
        "max_steps": max_steps,
        "learning_rate": learning_rate,
        "eval_iters": eval_iters,
        "n_emb": n_emb,
        "n_layer": n_layer,
        "n_head": n_head,
        "dropout": dropout,
        "vocab_size": vocab_size,
        "stoi": stoi,
        "itos": itos
    }


    # Salve o dicionário em um arquivo JSON
    with open(save_config_path, 'w') as f:
        json.dump(config, f)